In [1]:
import math
import pandas as pd
import numpy as np
from mesa import Agent
import random

In [2]:
class Pupil(Agent):
    
    """
    Define the agent type: pupils
        they have location information and travel behaviours;
        They will carry out some functions: choose destination, choose mode, etc
        
        Attributes:
            unique_id : the id of the agent
            hx,hy: the home location of the agent's home
            school: the agent's destination school
            # road_p: road choice probability distribution, e.g. road_p = [0.3,0.3,0.3,0.1]
            mode_p: mode choice probability distribution, e.g. mode_p = [0.1,0.9,0,0]
            perception: the agent's perception towards the COVID-19, from 1-5 , the perception increases
            health: agent's health condition, "s","i","r" representing the susecptibel, exposed, infected, reovered
    
    """
    
    def __init__(self,
                unique_id,
                model,
                hx,
                hy,
                school,  # road_p is deleted here
                mode_p,
                perception,
                health):
        ## define a new agent
        
        super().__init__(unique_id,model)
        self.breed = 'pupil' # define the agent type: pupil
        self.hx = hx
        self.hy = hy
        self.school = school
        #self.road_p = road_p
        self.mode_p = mode_p
        self.perception = perception
        self.health = health
        self.infection_time = 0  ## record the infection time
        self.time = 0
        
        
        ## define the mode choice and corresponding risk, simple version by considering the whole commute process as a discrete event
        self.modes = ['bus','school bus','private car','walk']
        # mode_risk = {'bus':0.03,'school bus':0.01,'private car':0.001,'walk':0.02}
        self.roads = {'bus':[7],'school bus':[6],'private car':[4,5],'walk':[1,2,3]}
        self.road_risk = {1:0.005,2:0.004,3:0.002,4:0.004,5:0.005,6:0.01,7:0.003}
        self.road_cell = {1:(50,1),2:(50,2),3:(50,3),4:(50,4),5:(50,5),6:(50,6),7:(50,7)}
        
        
        
        

        
           
        
    def infect(self):
        """Find close contacts and infect"""

        cellmates = self.model.grid.get_cell_list_contents([self.pos])       
        if len(cellmates) > 1:
            for other in cellmates:
                if (self.random.random() < self.model.trans_prob*((6-self.perception)/5)*((6-other.perception)/5)) and other.health =='s':                    
                    other.health = 'i'
                    other.infection_time = self.model.schedule.time
                    
                    
            
    

        
        
    def test(self):
        
        schools = {1:(15,15),2:(30,30)}
        self.model.grid.move_agent(self,schools[self.school])
        if self.health == 'i':
            self.infect()
        
        self.model.grid.move_agent(self,(self.hx,self.hy))
        
    
    def step1(self):
        ## move to the assigned road from home
        mode = np.random.choice(self.modes,size=None,p=self.mode_p)
            
        # assign the road
        road = np.random.choice(self.roads[mode])
        #self.model.grid.move_agent(self,road_cell[road])    
        # calculate the travel risk = risk * perception
            
        risk = self.model.road_risk * (6-self.perception)
        self.model.grid.move_agent(self,self.road_cell[road])
        
        ## if agent is susceptibel, it is possible to be infected 
        if self.health == 's':
            if random.random()<risk:
                self.health = 'i'
                self.infection_time = self.model.schedule.time
        
        
        # if it is infected, calculate whether it is recovered
        ## if agent is infected, it is possible to infect other agents.   
        if self.health == 'i':
            t = self.model.schedule.time-self.infection_time
            if t >= random.randint(28,56):          
                self.health = 'r'
            else:
                self.infect()
    
    def step2(self):
        
        ## move to school from the road
        
        schools = {1:(15,15),2:(30,30)}
        self.model.grid.move_agent(self,schools[self.school])
        if self.health == 'i':
            t = self.model.schedule.time-self.infection_time
            if t >= random.randint(28,56):          
                self.health = 'r'
            else:
                self.infect()
        if self.health == 's':
            risk = self.model.school_risk*(6-self.perception)/5
            if random.random() < risk:
                self.health = 'i'
                self.infection_time = self.model.schedule.time
            
    def step3(self):
        
        # move to the home from the road
        
        self.model.grid.move_agent(self,(self.hx,self.hy))
        if self.health == 'i':
                t = self.model.schedule.time-self.infection_time
                if t >= random.randint(28,56):          
                    self.health = 'r'
                else:
                    self.infect()
        
        
        if self.health == 's':
            risk = self.model.house_risk*(6-self.perception)/5
            if random.random() < risk:
                self.health = 'i'
                self.infection_time = self.model.schedule.time
        

    def step(self):
    
        
        """
        This step function will tell agents their agenda for one day
        
        First, detect if the agent is infected, this is for destination choosing;
        
        Then, choose the mode for this destination;
        
        The movement will happen by move function, and the process will include the transmission or infection
        
        Once arrive the destination, the agent will stay there and have another possible transmission or infection
      
        """
        self.time = self.time+1
        
        if self.time % 4 == 1:
            self.step1()
        if self.time % 4 == 2:
            self.step2()
        if self.time % 4 == 3:
            self.step1()
        if self.time % 4 == 0:
            self.step3()
        
        
        
        #self.assign_road()
        #self.school_activity()
        #self.go_home()
        #self.test()
   
    
    
            
            
            
        
    
        

In [3]:
class Household(Agent):
    """
    
    Now define the agnet type: Household members
    
    work: whether they need to go to the workplace, 0:stay home; 1:go outside
    
    hx,hy: house location
    
    workplace: if they need to go to the work place,there are 1,2,3 different workplaces
    
    mode_p: the mode choice probablility distribution, as Pupil
    
    perception: the covid-19 perception, from 1-5
    
    """
    def __init__(self, unique_id, model,hx, hy, work, workplace, mode_p, perception, health):
        
        super().__init__(unique_id,model)
        self.breed = 'household'
        self.work = work
        self.hx = hx
        self.hy = hy
        self.workplace =  workplace
        self.mode_p = mode_p
        self.perception = perception
        self.health = health
        
        self.infection_time = 0  ## record the infection time
        self.time = 0
        
        
        ## define the mode choice and corresponding risk, simple version by considering the whole commute process as a discrete event
        self.modes = ['bus','private car','walk']
        # mode_risk = {'bus':0.03,'school bus':0.01,'private car':0.001,'walk':0.02}
        self.roads = {'bus':[6,7,8,9],'private car':[4,5],'walk':[1,2,3]}
        self.road_risk = {1:0.005,2:0.004,3:0.002,4:0.004,5:0.005,6:0.01,7:0.003,8:0.002,9:0.001}
        self.road_cell = {1:(49,1),2:(49,2),3:(49,3),4:(49,4),5:(49,5),6:(49,6),7:(49,7),8:(49,8),9:(49,9)}
    
    def infect(self):
        """Find close contacts and infect"""

        cellmates = self.model.grid.get_cell_list_contents([self.pos])       
        if len(cellmates) > 1:
            for other in cellmates:
                if (self.random.random() < self.model.trans_prob*((6-self.perception)/5)*((6-other.perception)/5)) and other.health =='s':                    
                    other.health = 'i'
                    other.infection_time = self.model.schedule.time
                    
    
    def step1(self):
        if self.work == 1:
            ## move to the assigned road from home
            mode = np.random.choice(self.modes,size=None,p=self.mode_p)
            
            # assign the road
            road = np.random.choice(self.roads[mode])
            #self.model.grid.move_agent(self,road_cell[road])    
            # calculate the travel risk = risk * perception
            
            #risk = self.road_risk[road] * (6-self.perception)
            risk = self.model.road_risk*(6-self.perception)
            self.model.grid.move_agent(self,self.road_cell[road])
        
            ## if agent is susceptibel, it is possible to be infected 
            if self.health == 's':
                if random.random()<risk:
                    self.health = 'i'
                    self.infection_time = self.model.schedule.time
        
        
            # if it is infected, calculate whether it is recovered
            ## if agent is infected, it is possible to infect other agents.   
            if self.health == 'i':
                t = self.model.schedule.time-self.infection_time
                if t >= random.randint(28,56):          
                    self.health = 'r'
                else:
                    self.infect()
    
    def step2(self):
        
        ## move to workplaces from the road, if they are healthy, they may be infected by the environment.
        
        workplaces = {1:(10,1),2:(7,7),3:(5,5)}
        if self.work == 1:
            self.model.grid.move_agent(self,workplaces[self.workplace])
            if self.health == 'i':
                t = self.model.schedule.time-self.infection_time
                if t >= random.randint(28,56):          
                    self.health = 'r'
                else:
                    self.infect()
                
            if self.health == 's':
                risk = self.model.workplace_risk*(6-self.perception)/5
                if random.random() < risk:
                    self.health = 'i'
                    self.infection_time = self.model.schedule.time
            
    def step3(self):
        
        # move to the home from the road
        
        self.model.grid.move_agent(self,(self.hx,self.hy))
        if self.health == 'i':
                t = self.model.schedule.time-self.infection_time
                if t >= random.randint(28,56):          
                    self.health = 'r'
                else:
                    self.infect()
        if self.health == 's':
            risk = self.model.house_risk*(6-self.perception)/5
            if random.random() < risk:
                self.health = 'i'
                self.infection_time = self.model.schedule.time
        
        
        
        

    def step(self):
    
        
        """
        This step function will tell agents their agenda for one day
        
        First, detect if the agent is infected, this is for destination choosing;
        
        Then, choose the mode for this destination;
        
        The movement will happen by move function, and the process will include the transmission or infection
        
        Once arrive the destination, the agent will stay there and have another possible transmission or infection
      
        """
        self.time = self.time+1
        
        if self.time % 4 == 1:
            self.step1()
        if self.time % 4 == 2:
            self.step2()
        if self.time % 4 == 3:
            self.step1()
        if self.time % 4 == 0:
            self.step3()
        
        
        
        
        
        
        
        

In [4]:
class School(Agent):
    """
    
    Now define the agnet type: school
    
    
    
    """
    def __init__(self, unique_id, model,hx,hy):
        
        super().__init__(unique_id,model)
        self.breed = 'school'
        self.hx = hx
        self.hy = hy
        self.health = 's'
        self.perception = 1
        
class Workplace(Agent):
    """
    
    Now define the agnet type: school
    
    
    
    """
    def __init__(self, unique_id, model,hx,hy):
        
        super().__init__(unique_id,model)
        self.breed = 'workplace'
        self.hx = hx
        self.hy = hy     
        self.health = 's'
        self.perception = 1
        

        
        
        
        
        
        
        
        

In [6]:
from mesa import Model
from mesa.datacollection import DataCollector
from mesa.time import SimultaneousActivation
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector


def total_inf(model):
    infected_agent = [a for a in model.schedule.agents if a.health == 'i']
    return len(infected_agent)


def pupil_count(model):
    """return number of rich agents"""

    infected_Pupil = [a for a in model.schedule.agents if type(a) == Pupil and a.health == 'i']
    return len(infected_Pupil)

def household_inf(model):
    infected_Household = [a for a in model.schedule.agents if type(a) == Household and a.health == 'i']
    return len(infected_Household)

def home_inf(model):
    home_inf = 0
    for agent in model.schedule.agents:
        if agent.health == 'i' and agent.pos == (agent.hx, agent.hy):
            home_inf += 1
    return home_inf

def school_inf(model):
    school_inf = 0
    schools = {1: (15, 15), 2: (30, 30)}
    for agent in model.schedule.agents:
        if agent.breed == 'pupil':
            if agent.health == 'i' and agent.pos == schools[agent.school]:
                school_inf += 1
    return school_inf

def workplace_inf(model):
    workplace_inf = 0
    workplaces = {1:(10,1),2:(7,7),3:(5,5)}
    for agent in model.schedule.agents:
        if agent.breed == 'household':
            if agent.health == 'i' and agent.pos == workplaces[agent.workplace]:
                workplace_inf += 1
    return workplace_inf

def road_inf(model):
    road_inf = 0
    road_inf = total_inf(model) - home_inf(model) - workplace_inf(model) - school_inf(model) 
    return road_inf    


class SimModel(Model):
    
    """
    The model will give a basic enviornment paramter configuration.
    And generate the grid, agents.
    """
    
    def __init__(self,
                 height = 51,
                 width = 51,
                 road_risk = 0.2,
                 house_risk = 0.2,
                 school_risk = 0.2,
                 workplace_risk = 0.2,
                 trans_prob = 0.5
                ):
        super().__init__()
        self.height = height
        self.width = width
        self.road_risk = road_risk
        self.house_risk = house_risk
        self.school_risk = school_risk
        self.workplace_risk = workplace_risk
        self.trans_prob = trans_prob
        
        self.grid = MultiGrid(height,width,torus=False)
        
        self.iteration = 0
    
    
        self.schedule = SimultaneousActivation(self)
        
        # collect the data
        model_reporters = {
            "Total_infection": total_inf,
            "Pupil infection": pupil_count,
            "Household Infection": household_inf,
            "Home infection": home_inf,
            "School infection": school_inf,
            "Workplace infection": workplace_inf,
            "Route infection": road_inf
        }
        
        self.datacollector = DataCollector(model_reporters=model_reporters,agent_reporters=None)
    
    
        # generate the agents from the file
        AgentParaDataFrame = pd.read_csv('pupil.csv')
        for row in range(0, AgentParaDataFrame.shape[0]):
            agent = Pupil(
                AgentParaDataFrame.iloc[row][0],
                self,
                AgentParaDataFrame.iloc[row][2],
                AgentParaDataFrame.iloc[row][3],
                AgentParaDataFrame.iloc[row][4],
                (AgentParaDataFrame.iloc[row][5],AgentParaDataFrame.iloc[row][6],AgentParaDataFrame.iloc[row][7],AgentParaDataFrame.iloc[row][8]),
                AgentParaDataFrame.iloc[row][9],
                AgentParaDataFrame.iloc[row][10]
                )
            self.schedule.add(agent)
            self.grid.place_agent(agent,(agent.hx,agent.hy))
            #self.running = True
            
         
        HouseholdPara = pd.read_csv('household.csv')
        for row in range(0,HouseholdPara.shape[0]):
            agent = Household(
                HouseholdPara.iloc[row][0]+30,
                self,
                HouseholdPara.iloc[row][2],
                HouseholdPara.iloc[row][3],
                HouseholdPara.iloc[row][4],
                HouseholdPara.iloc[row][5],
                (HouseholdPara.iloc[row][6],HouseholdPara.iloc[row][7],HouseholdPara.iloc[row][8]),
                HouseholdPara.iloc[row][9],
                HouseholdPara.iloc[row][10]
                )
            self.schedule.add(agent)
            self.grid.place_agent(agent,(agent.hx,agent.hy))
    

        workplace1 = Workplace(600,self,10,1)
        workplace2 = Workplace(601,self,7,7)
        workplace3 = Workplace(602,self,5,5)
        workplaces = [workplace1, workplace2, workplace3]
        for agent in workplaces:
            self.schedule.add(agent)
            self.grid.place_agent(agent,(agent.hx,agent.hy))
        
        
        school1 = School(701,self,15,15)
        school2 = School(702,self,30,30)
        schools = [school1,school2]
        for agent in schools:
            self.schedule.add(agent)
            self.grid.place_agent(agent,(agent.hx,agent.hy))
        
        self.running = True
    
    def step(self):
        
        #if self.schedule.time % 4 == 0:
        self.datacollector.collect(self)
        self.schedule.step()
        
        
    

In [ ]:
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.modules import ChartModule
from mesa.visualization.UserParam import UserSettableParameter
import nest_asyncio
nest_asyncio.apply()


def agent_portrayal(agent):
    
    
    portrayal = {}
    
    if agent.breed == 'pupil':
        portrayal['Shape'] = 'children.png'
        portrayal["scale"] = 0.9
        portrayal["Layer"] = 1
        
    if agent.breed == 'household':
        portrayal['Shape'] = 'worker.png'
        portrayal["scale"] = 0.9
        portrayal["Layer"] = 1
        
    if agent.breed == 'school':
        portrayal['Shape'] = 'school.png'
        portrayal["scale"] = 0.9
        portrayal["Layer"] = 2
        
    if agent.breed == 'workplace':
        portrayal['Shape'] = 'workplace.png'
        portrayal["scale"] = 0.9
        portrayal["Layer"] = 2
    
    
    if agent.health == 's':
        portrayal['Color'] = 'green'
    if agent.health == 'i':
        portrayal['Color'] = 'red'
    if agent.health == 'r':
        portrayal['Color'] = 'gray'

    
    
    
    
  
    return portrayal


grid = CanvasGrid(agent_portrayal, 52, 52, 500,500)


chart1 = ChartModule(
    [
        {"Label":"Total_infection","Color":"red"},
        {"Label":"Pupil infection","Color":"green"},
        {"Label":"Household Infection","Color":"yellow"}
        
    ]
    )


chart2 = ChartModule(
    [
        {"Label":"Home infection","Color":"blue"},
        {"Label":"School infection","Color":"grey"},
        {"Label":"Workplace infection","Color":"black"},
        {"Label":"Road infection","Color":"orange"}
    ]
    )
model_params = {
    "height" : 52,
    "width" : 52,
    "road_risk" : UserSettableParameter("slider","the risk in the road",0.2,0.01,0.99,0.01),
    "house_risk" : UserSettableParameter("slider","the risk in the house",0.2,0.01,0.99,0.01),
    "school_risk" : UserSettableParameter("slider","the risk in the school",0.2,0.01,0.99,0.01),
    "workplace_risk" : UserSettableParameter("slider","the risk in the workplace",0.2,0.01,0.99,0.01),
    "trans_prob" : UserSettableParameter("slider","the risk of an effective contact",0.5,0.01,0.99,0.01)
}

server = ModularServer(SimModel,
                       [chart1,chart2,grid],
                       "SimIMBP",
                       model_params)
server.port = 8833 # The default
server.launch()

Interface starting at http://127.0.0.1:8833
Socket opened!
{"type":"reset"}
{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":10}
{"type":"get_step","step":11}
{"type":"get_step","step":12}
{"type":"get_step","step":13}
{"type":"get_step","step":14}
{"type":"get_step","step":15}
{"type":"get_step","step":16}
{"type":"get_step","step":17}
{"type":"get_step","step":18}
{"type":"get_step","step":19}
{"type":"get_step","step":20}
{"type":"get_step","step":21}
{"type":"get_step","step":22}
{"type":"get_step","step":23}
{"type":"get_step","step":24}
{"type":"get_step","step":25}
{"type":"get_step","step":26}
{"type":"get_step","step":27}
{"type":"get_step","step":28}
{"type":"get_step","step":29}
{"type":"get_step","step":30}
{"type":"get_step","step":31}
{"t

In [ ]:
model = SimModel(51,51)
for i in range(10):
    model.step()

In [ ]:
len([a for a in model.schedule.agents if type(a)== Pupil])

In [ ]:
len([a for a in model.schedule.agents if type(a)== Household])

In [ ]:
len([a for a in model.schedule.agents if type(a)== School])

In [ ]:
len([a for a in model.schedule.agents if type(a)== Workplace])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
agent_counts = np.zeros((model.grid.width, model.grid.height))
for cell in model.grid.coord_iter():
    cell_content, x, y = cell
    agent_count = len(cell_content)
    agent_counts[x][y] = agent_count
plt.imshow(agent_counts, interpolation='nearest')
plt.colorbar()


In [ ]:
## add animation, server
## adjust the order

In [ ]:
model.datacollector.get_model_vars_dataframe()